# Lecture 11 – Imputation

### DATA 2201, Fall 2024

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_inline.backend_inline import set_matplotlib_formats
from IPython.display import display, IFrame, HTML
from pathlib import Path

import plotly
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

pd.options.plotting.backend = "plotly"
plt.rcParams["figure.figsize"] = (10, 5)

In [ ]:
# Import several helper functions for this lecture, 
#  Used for plotting and permutation tests
from lec11_utils import *

### Agenda

- Review: Missingness mechanisms.
- Identifying missingness mechanisms in data.
    - How do we decide between MCAR and MAR using a permutation test?
    - The Kolmogorov-Smirnov test statistic.
- Imputation.
    - Mean imputation.
    - Probabilistic imputation.

## Review: Missingness mechanisms

### Flowchart

A good strategy is to assess missingness in the following order.

<center><b>Missing by design (MD)</b></center>
<center><i>Can I determine the missing value exactly by looking at the other columns?</i> 🤔</center>
<center> ⬇️ </center>

<center><b>Not missing at random (NMAR)</b></center>
<center><i>Is there a good reason why the missingness depends on the values themselves?</i> 🤔</center>
<center> ⬇️ </center>

<center><b>Missing at random (MAR)</b></center>
<center><i>Do other columns tell me anything about the likelihood that a value is missing? </i>🤔</center>
<center> ⬇️ </center>

<center><b>Missing completely at random (MCAR)</b></center>
<center><i>The missingness must not depend on other columns or the values themselves. </i>😄</center>

### Quick Check

<small>_Taken from the Winter 2023 DSC 80 Midterm Exam._</small>
    
The DataFrame `tv_excl` contains all of the information we have for TV shows that are only available for streaming on a single streaming service.
    
<center>
    <img src="imgs/tv-excl.png" width=60%>
</center>
    
    
Given no other information other than a TV show’s `"Title"` and `"IMDb"` rating, what is the most likely missingness mechanism of the `"IMDb"` column?

- A. Missing by design

- B. Not missing at random

- C. Missing at random

- D. Missing completely at random

<br>
<details><summary>Solution</summary>
    The answer we were looking for is not missing at random (NMAR). As we saw repeatedly in lecture 10, in cases where all we have access to is a single column with missing values, potentially with other unrelated columns (like `"Title"` here), the best explanation is that there is some inherent reason as to why the values in the column with missing values are missing. Here, a reasonable interpretation is that the `"IMDb"` scores that are missing are likely to come from worse TV shows, and so lower scores are more likely to be missing. Think about it like this – if a TV show is really great, presumably more people would know about it, and it would be rated. If a TV show wasn’t as good and wasn’t as popular, it is more likely to be ignored.

However, partial credit was awarded to those who answered missing completely at random.
</details>

### Quick Check

<small>_Taken from the Winter 2023 DSC 80 Midterm Exam._</small>
        
<center>
    <img src="imgs/tv-excl.png" width=60%>
</center>
    
    
Now, suppose we discover that the median `"Rotten Tomatoes"` rating among TV shows with a missing `"IMDb"` rating is a 13, while the median `"Rotten Tomatoes"` rating among TV shows with a present `"IMDb"` rating is a 52.

Given this information, what is the most likely missingness mechanism of the `"IMDb"` column?

- A. Missing by design

- B. Not missing at random

- C. Missing at random

- D. Missing completely at random

<br>
<details><summary>Solution</summary>
    The problem tells us that the distribution of `"Rotten Tomatoes"` when `"IMDb"` is missing (mean 13) is very different from the distribution of `"Rotten Tomatoes"` when `"IMDb"` is not missing (mean 52). As such, the missingness of `"IMDb"` appears to depend on `"Rotten Tomatoes"`, and so the most likely missingness mechanism is missing at random.
</details>

### Review: Example - Heights data 

Last lecture, we saw an example using height's data for child, mother and father. 

We saw that if we simulated MAR data, that there appeared to be a difference in the distribution between the non and non-null `'child'` distribution and `'father'` distribution. 

However, the difference in means for the distributions was very small. 

Therefore, if we ran a permutation test with the difference in means as our test statistic, we would fail to reject the null.
    - **Using just the difference in means, it is hard to tell these two distributions apart.**

In [ ]:
heights_path = Path('data') / 'midparent.csv'
heights = pd.read_csv(heights_path).rename(columns={'childHeight': 'child'})[['father', 'mother', 'gender', 'child']]

def make_missing(r):
    rand = np.random.uniform() # Random real number between 0 and 1.
    if r['father'] > 72 and rand < 0.5:
        return np.NaN
    elif r['gender'] == 'female' and rand < 0.3:
        return np.NaN
    else:
        return r['child']
    
heights_mar = heights.copy()
heights_mar['child'] = heights_mar.apply(make_missing, axis=1)
heights_mar['child_missing'] = heights_mar['child'].isna()

## The Kolmogorov-Smirnov test statistic

### Recap: Permutation tests

- Permutation tests help decide whether **two samples look like they were drawn from the same population distribution**.

- In a permutation test, we simulate data under the null by **shuffling** either group labels or numerical features.
    - In effect, this **randomly assigns individuals to groups**.

- If the two distributions are **numerical**, we've used as our test statistic the **difference in group means or medians**.

- If the two distributions are **categorical**, we've used as our test statistic the **total variation distance (TVD)**.

### Difference in means

The difference in means works well in some cases. Let's look at one such case.

Below, we artificially generate two numerical datasets.

In [ ]:
np.random.seed(42) # So that we get the same results each time (for lecture).

N = 1000 # Number of samples for each distribution.

# Distribution 'A'.
distr1 = pd.Series(np.random.normal(0, 1, size=N // 2))

# Distribution 'B'.
distr2 = pd.Series(np.random.normal(3, 1, size=N // 2))

data = pd.concat([distr1, distr2], axis=1, keys=['A', 'B']).unstack().reset_index().drop('level_1', axis=1)
data = data.rename(columns={'level_0': 'group', 0: 'data'})

meanA, meanB = data.groupby('group')['data'].mean().round(7).tolist()
create_kde_plotly(data, 'group', 'A', 'B', 'data', f'mean of A: {meanA}<br>mean of B: {meanB}')

### Different distributions with the same mean

Let's generate two distributions that look very different but have the same mean.

In [ ]:
np.random.seed(42) # So that we get the same results each time (for lecture).

N = 1000 # Number of samples for each distribution.

# Distribution 'A'.
a = pd.Series(np.random.normal(0, 1, size=N//2))
b = pd.Series(np.random.normal(4, 1, size=N//2))
distr1 = pd.concat([a,b], ignore_index=True)

# Distribution 'B'.
distr2 = pd.Series(np.random.normal(distr1.mean(), distr1.std(), size=N))

data = pd.concat([distr1, distr2], axis=1, keys=['A', 'B']).unstack().reset_index().drop('level_1', axis=1)
data = data.rename(columns={'level_0': 'group', 0: 'data'})

meanA, meanB = data.groupby('group')['data'].mean().round(7).tolist()
create_kde_plotly(data, 'group', 'A', 'B', 'data', f'mean of A: {meanA}<br>mean of B: {meanB}')

In this case, if we use the difference in means as our test statistic in a permutation test, we will fail to reject the null that the two distributions are different.

In [ ]:
n_repetitions = 500
shuffled = data.copy()

diff_means = []
for _ in range(n_repetitions):
    
    # Shuffling the values, while keeping the group labels in place.
    shuffled['data'] = np.random.permutation(shuffled['data'])
    
    # Computing and storing the absolute difference in means.
    diff_mean = shuffled.groupby('group')['data'].mean().diff().abs().iloc[-1]
    diff_means.append(diff_mean)

In [ ]:
observed_diff = data.groupby('group')['data'].mean().diff().abs().iloc[-1]
fig = px.histogram(pd.DataFrame(diff_means), x=0, nbins=50, histnorm='probability', 
                   title='Empirical Distribution of the Absolute Difference in Means')
fig.add_vline(x=observed_diff, line_color='red', line_width=1, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed Absolute Difference in Means = {round(observed_diff, 2)}</span>',
                   x=2 * observed_diff, showarrow=False, y=0.07)

In [ ]:
# The computed p-value is fairly large.
np.mean(np.array(diff_means) >= observed_diff)

### Telling numerical distributions apart

- The difference in means only works as a test statistic in permutation tests **if the two distributions have similar shapes**.
    - It tests to see if one is a shifted version of the other.

- We need a better test statistic to differentiate between numerical distributions with different shapes.

- In other words, we need a **distance** metric between numerical.
    - The TVD is a distance metric between categorical distributions.

In [ ]:
create_kde_plotly(data, 'group', 'A', 'B', 'data', f'mean of A: {meanA}<br>mean of B: {meanB}')

### The Kolmogorov-Smirnov test statistic

- The K-S test statistic measures the similarity between two distributions.
- It is defined in terms of the **cumulative distribution function (CDF)** of a given distribution.
    - If $f(x)$ is a distribution, then the CDF $F(x)$ is the proportion of values in distribution $f$ that are less than or equal to $x$.
- The K-S statistic is roughly defined as the **largest difference between two CDFs**.
<center><img src=./imgs/KS2_Example.png width=50%></center>

#### Aside: Cumulative distribution functions

First, some setup.

In [ ]:
fig1 = create_kde_plotly(data, 'group', 'A', 'B', 'data', f'Distributions of A and B')

# Think about what this function is doing!
def create_cdf(group):
    return data.loc[data['group'] == group, 'data'].value_counts(normalize=True).sort_index().cumsum()

fig2 = go.Figure()

fig2.add_trace(
    go.Scatter(x=create_cdf('A').index, y=create_cdf('A'), name='CDF of A')
)

fig2.add_trace(
    go.Scatter(x=create_cdf('B').index, y=create_cdf('B'), name='CDF of B')
)

fig2.update_layout(title='CDFs of A and B')

from plotly.subplots import make_subplots

for i in range(2):
    fig2.data[i]['marker']['color'] = fig1.data[i]['marker']['color']
    fig2.data[i]['showlegend'] = False
    
fig = make_subplots(rows=1, cols=2, subplot_titles=['Distributions', 'CDFs'])
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig1.data[1], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)
fig.add_trace(fig2.data[1], row=1, col=2)
fig.update_layout(width=1000, height=400);

Let's look at the CDFs of our two synthetic distributions.

In [ ]:
fig

### The K-S statistic in Python

Fortunately, **we don't need to calculate the K-S statistic ourselves**! Python can do it for us (and you can use this pre-built version in all assignments).

In [ ]:
from scipy.stats import ks_2samp

In [ ]:
display(data.head(3))
display(data.tail(3))

In [ ]:
ks_2samp?

In [ ]:
observed_ks = ks_2samp(data.loc[data['group'] == 'A', 'data'], 
                       data.loc[data['group'] == 'B', 'data']).statistic
observed_ks

We don't know if this number is big or small. We need to run a permutation test!

In [ ]:
n_repetitions = 500
shuffled = data.copy()

ks_stats = []
for _ in range(n_repetitions):
    
    # Shuffling the data.
    shuffled['data'] = np.random.permutation(shuffled['data'])
    
    # Computing and storing the K-S statistic.
    groups = shuffled.groupby('group')['data']
    ks_stat = ks_2samp(groups.get_group('A'), 
                       groups.get_group('B')).statistic
    ks_stats.append(ks_stat)
    
ks_stats[:10]

In [ ]:
fig = px.histogram(pd.DataFrame(ks_stats), x=0, nbins=50, histnorm='probability', 
                   title='Empirical Distribution of the K-S Statistic')
fig.add_vline(x=observed_ks, line_color='red', line_width=1, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed KS = {round(observed_ks, 2)}</span>',
                   x=0.8 * observed_ks, showarrow=False, y=0.16)

fig.update_layout(xaxis_range=[0, 0.2])
fig.update_layout(yaxis_range=[0, 0.2])

In [ ]:
np.mean(np.array(ks_stats) >= observed_ks)

We were able to differentiate between the two distributions using the K-S test statistic!

### `ks_2samp`

* `scipy.stats.ks_2samp` actually returns **both** the statistic **and** a p-value.
* The p-value is calculated using the permutation test we just performed!

In [ ]:
ks_2samp(data.loc[data['group'] == 'A', 'data'], data.loc[data['group'] == 'B', 'data'])

### Difference in means vs. K-S statistic

- The K-S statistic measures the difference between two numerical distributions.

- It **does not** quantify if one is larger than the other on average, so there are times we still need to use the difference in means.


- Strategy: Always plot the two distributions you are comparing.
    - If the distributions have similar shapes but are centered in different places, use the difference in means (or absolute difference in means).
    - If your alternative hypothesis involves a "direction" (i.e. smoking weights were are on average than non-smoking weights), use the difference in means.
    - If the distributions have different shapes but roughly the same center, and your alternative hypothesis is simply that the two distributions are different, use the K-S statistic.

### Example: Heights -  Missingness of `'child'` heights on `'father'`'s heights (MAR)

In [ ]:
heights_mar['child_missing'] = heights_mar['child'].isna()
create_kde_plotly(heights_mar[['child_missing', 'father']], 
                  'child_missing', True, False, 'father',
                  "Father's Height by Missingness of Child Height (MAR example)")

- The above picture shows us that missing `'child'` heights tend to come from taller `'father'`s heights.

- To determine whether the two distributions are significantly different, we must use a permutation test. This time, the difference in means is not a good choice, since the centers are similar but the shapes are different.

### Performing the test

In [ ]:
heights_mar

In [ ]:
ks_2samp(heights_mar.query('child_missing')['father'], heights_mar.query('not child_missing')['father'])

**The p-value is very small, so we conclude that the child height is MAR, conditional on the father's height.**

## Handling missing values

### What do we do with missing data?

- Suppose we are interested in a dataset $Y$. 
- We get to **observe** $Y_{obs}$, while the rest of the dataset, $Y_{mis}$, is **missing**.
- Issue: $Y_{obs}$ may look quite different than $Y$.
    - The mean and other measures of central tendency may be different.
    - The variance may be different.
    - The correlations between variables may be different.

### Solution 1: Dropping missing values

- If the data are MCAR (missing completely at random), then dropping the missing values entirely doesn't significantly change the data.
    - For instance, the mean of the dataset post-dropping is an unbiased estimate of the true mean.
    - This is because MCAR data is a **random sample** of the full dataset.
    - From DSC 10, we know that random samples tend to resemble the larger populations they are drawn from.

- **If the data are not MCAR, then dropping the missing values will introduce bias.**
    - MCAR is rare!
    - For instance, suppose we asked people "How much do you give to charity?" People who give little are less likely to respond, so the average response is **biased high**.

#### Listwise deletion

- _Listwise deletion_ is the act of dropping entire rows that contain missing values.
- Issue: This can delete perfectly good data in other columns for a given row.
    - Improvement: Drop missing data only when working with the column that contains missing data.

To illustrate, let's generate two datasets with missing `'child'` heights – one in which the heights are MCAR, and one in which they are MAR dependent on `'gender'` only.

**In practice, you'll have to run permutation tests to determine the likely missingness mechanism first!**

In [ ]:
np.random.seed(42) # So that we get the same results each time (for lecture).
heights_mcar = make_mcar(heights, 'child', pct=0.5)
heights_mar = make_mar_on_cat(heights, 'child', 'gender', pct=0.5)



Below, we compute the means and standard deviations of the `'child'` column in all three datasets. Remember, `.mean()` and `.std()` ignore missing values.

In [ ]:
multiple_describe({
    'Original': heights,
    'MCAR': heights_mcar,
    'MAR': heights_mar
})

Observations:

- The `'child'` mean (and SD) in the MCAR dataset is very close to the true `'child'` mean (and SD).

- The `'child'` mean in the MAR dataset is biased **high**.

### Solution 2: Imputation

**Imputation** is the act of filling in missing data with plausable values. Ideally, imputation:

* is quick and easy to do.
* shouldn't introduce bias into the dataset.

These are hard to do at the same time!

#### Kinds of imputation

- There are three main types of imputation, two of which we will focus on today:

    - **Imputation with a single value: mean, median, mode.**
    - Imputation with a single value, using a model: regression, kNN.
    - **Probabilistic imputation by drawing from a distribution.**

- Each has upsides and downsides, and **each works differently with different types of missingness**.

## Mean imputation



- Mean imputation is the act of filling in missing values in a column with the mean of the observed values in that column.
- This strategy:
    - 👍 Preserves the mean of the observed data, for all types of missingness.
    - 👎 Decreases the variance of the data, for all types of missingness.
    - 👎 Creates a biased estimate of the true mean when the data are not MCAR.

#### Example: Mean imputation in the MCAR `heights` dataset

Let's look at two distributions:
- The distribution of the `'child'` column in `heights`, where we have all the data.
- The distribution of the `'child'` column in `heights_mcar`, where some values are MCAR.

In [ ]:
# Look in util.py to see how multiple_kdes is defined.
multiple_kdes({'Original': heights, 'MCAR, Unfilled': heights_mcar})

- Since the `'child'` heights are MCAR, the <span style='color:rgb(217,95,2)'><b> orange distribution, in which some values are missing</b></span>, has roughly the same shape as the <span style='color:rgb(27,158,119)'><b>turquoise distribution, which has no missing values</b></span>.

### Mean imputation of MCAR data

Let's fill in missing values in `heights_mcar['child']` with the mean of the observed `'child'` heights in `heights_mcar['child']`.

In [ ]:
heights_mcar['child'].head()

In [ ]:
heights_mcar_mfilled = heights_mcar.fillna(heights_mcar['child'].mean())
heights_mcar_mfilled['child'].head()

In [ ]:
df_map = {'Original': heights, 'MCAR, Unfilled': heights_mcar, 'MCAR, Mean Imputed': heights_mcar_mfilled}
multiple_describe(df_map)

Observations:

- The mean of the imputed dataset is the same as the mean of the subset of heights that aren't missing (which is close to the true mean).

- The standard deviation of the imputed dataset smaller than that of the other two datasets. **Why?**



Let's visualize all three distributions: the original, the MCAR heights with missing values, and the mean-imputed MCAR heights.

In [ ]:
multiple_kdes(df_map)

**Takeaway**: When data are MCAR and you impute with the mean:
- The mean of the imputed dataset is an **unbiased estimator** of the true mean.
- The variance of the imputed dataset is smaller than the variance of the full dataset.
    - Mean imputation tricks you into thinking your data are more reliable than they are!

#### Example: Mean imputation in the MAR `heights` dataset

- When data are MAR, mean imputation leads to biased estimates of the mean across groups.

- The bias may be different in different groups.
    - For example: If the missingness depends on gender, then different genders will have differently-biased means.
    - The overall mean will be biased towards one group.

- Again, let's look at two distributions:
    - The distribution of the `'child'` column in `heights`, where we have all the data.
    - The distribution of the `'child'` column in `heights_mar`, where some values are MAR.

In [ ]:
multiple_kdes({'Original': heights, 'MAR, Unfilled': heights_mar})

The distributions are not very similar!

Remember that in reality, you won't get to see the <span style='color:rgb(27,158,119)'><b>turquoise distribution, which has no missing values</b></span> – instead, you'll try to recreate it, using your sample with missing values.

### Mean imputation of MAR data

Let's fill in missing values in `heights_mar['child']` with the mean of the observed `'child'` heights in `heights_mar['child']` and see what happens.

In [ ]:
heights_mar['child'].head()

In [ ]:
heights_mar_mfilled = heights_mar.fillna(heights_mar['child'].mean())
heights_mar_mfilled['child'].head()

In [ ]:
df_map = {'Original': heights, 'MAR, Unfilled': heights_mar, 'MAR, Mean Imputed': heights_mar_mfilled}
multiple_describe(df_map)

Note that the latter two means are biased **high**.



Let's visualize all three distributions: the original, the MAR heights with missing values, and the mean-imputed MAR heights.

In [ ]:
multiple_kdes(df_map)

Since the sample with MAR values was already biased high, mean imputation kept the sample biased – it did not bring the data **closer to the data generating process**.

With our single mean imputation strategy, the resulting female mean height is biased quite high.

In [ ]:
pd.concat([
    heights.groupby('gender')['child'].mean().rename('Original'),
    heights_mar.groupby('gender')['child'].mean().rename('MAR, Unfilled'),
    heights_mar_mfilled.groupby('gender')['child'].mean().rename('MAR, Mean Imputed')
], axis=1).T

### Within-group (conditional) mean imputation

* **Improvement**: Since MAR data are MCAR within each group, we can perform group-wise mean imputation.
    - In our case, since the missingness of `'child'` is dependent on `'gender'`, we can impute separately for each `'gender'`.
    - For instance, if there is a missing `'child'` height for a `'female'` child, impute their height with the mean observed `'female'` height.

- With this technique, the overall mean remains unbiased, as do the within-group means.

- Like with "single" mean imputation, the variance of the dataset is reduced.

### `transform` returns!

- In MAR data, imputation by the overall mean gives a biased estimate of the mean of each group. 
- To obtain an unbiased estimate of the mean within each group, impute using the mean within each group.
- To perform an operation separately to each gender, we `groupby('gender')` and use the `transform` method.

In [ ]:
def mean_impute(s):
    return s.fillna(s.mean())

heights_mar_cond = heights_mar.groupby('gender')['child'].transform(mean_impute).to_frame()
heights_mar_cond['child'].head()

In [ ]:
df_map['MAR, Conditional Mean Imputed'] = heights_mar_cond
multiple_kdes(df_map)

The <span style='color:rgb(231,41,138)'><b>pink distribution</b></span> does a slightly better job of approximating the <span style='color:rgb(27,158,119)'><b>turquoise distribution</b></span> than the <span style='color:rgb(117,112,179)'><b>purple distribution</b></span>, but not by much.

### Conclusion: Imputation with single values

- Imputing missing data in a column with the mean of the column:
    - faithfully reproduces the mean of the observed dataset,
    - reduces the variance, and
    - biases relationships between the column and other columns if the data are not MCAR.
    
- The same is true with other statistics (e.g. median and mode).

## Probabilistic imputation

### Imputing missing values using distributions

- So far, each missing value in a column has been filled in with a constant value.
    - This creates "spikes" in the imputed distributions.

- **Idea**: We can **probabilistically** impute missing data from a distribution.
    - We can fill in missing data by drawing from the distribution of the **non-missing** data.
    - There are 5 missing values? Pick 5 values from the data that aren't missing.
     - How? Using `np.random.choice` or `.sample`.

- If the data are MCAR, then sample from the entire column of present values. If the data are MAR on some categorical column, then sample from the present values separately for each category.

### Example: Probabilistic imputation in the MAR `heights` dataset

Let's use `transform` to call `prob_impute` separately on each `'gender'`.

In [ ]:
def prob_impute(s):
    s = s.copy()
    
    # Step 1: Find the number of missing child heights for that gender.
    num_null = s.isna().sum()
    
    # Step 2: Sample num_null observed child heights for that gender.
    fill_values = np.random.choice(s.dropna(), num_null)
    
    # Step 3: Fill in missing values and return ser.
    s[s.isna()] = fill_values
    return s

In [ ]:
heights_mar_pfilled = heights_mar.copy()
heights_mar_pfilled['child'] = (
    heights_mar
    .groupby('gender')
    ['child']
    .transform(prob_impute)
)
heights_mar_pfilled['child'].head()

In [ ]:
df_map['MAR, Conditionally Probabilistically Imputed'] = heights_mar_pfilled
multiple_kdes(df_map)

The <b><span style="color:#67a61f">green distribution (conditional probabilistic imputation)</span></b> does the best job of approximating the <b><span style="color:#1c9e76">turqoise distribution (the full dataset with no missing values)</span></b>!

_Remember that the graph above is interactive – you can hide/show lines by clicking them in the legend._

### Observations

- With this technique, the missing values were filled in with observed values in the dataset.

- If a value was never observed in the dataset, it will never be used to fill in a missing value.
    - For instance, if the observed heights were 68, 69, and 69.5 inches, we will never fill a missing value with 68.5 inches even though it's a perfectly reasonable height.

- Solution? Create a histogram (with `np.histogram`) to bin the data, then sample from the histogram.
    - See Lab 5, Question 6.

### Randomness

- Unlike mean imputation, probabilistic imputation is **random** – each time you run the cell in which imputation is performed, the results could be different.

- If we're interested in estimating some population **parameter** given our (incomplete) sample, it's best not to rely on just a single random imputation.

- **Multiple imputation**: Generate multiple imputed datasets and aggregate the results!
    - Similar to bootstrapping.

## Summary

### Summary of imputation techniques

* Listwise deletion.
* Mean imputation.
* Group-wise (conditional) mean imputation.
* Probabilistic imputation.
* Multiple imputation.

### Summary: Listwise deletion

* Procedure: `df = df.dropna()`.
* If data are MCAR, listwise deletion doesn't change most summary statistics (mean, median, SD) of the data.

### Summary: Mean imputation 

* Procedure: `df[col] = df[col].fillna(df[col].mean())`.
* If data are MCAR, the resulting mean is an unbiased estimate of the true mean, but the variance is too low.
* Analogue for categorical data: imputation with the mode.

### Summary: Conditional mean imputation

* Procedure: for a column `c1`, conditional on a second categorical column
`c2`:

```py
means = df.groupby('c2').mean().to_dict()
imputed = df['c1'].apply(lambda x: means[x] if np.isnan(x) else x)
```

* If data are MAR, the resulting mean is an unbiased estimate of the true mean, but the variance is too low.
* This increases correlations between the columns.
* If the column with missing values were dependent on *more than one* column, we can use linear regression to predict the missing value.

### Summary: Probabilistic imputation

* Procedure: draw from the distribution of **observed data** to fill in missing values.
* If data are MCAR, the resulting mean and variance are unbiased estimates of the true mean and variance.
* Extending to the MAR case: draw from **conditional empirical distributions**.
    - If data are conditional on a single categorical column `c2`, apply the MCAR procedure to the groups of `df.groupby(c2)`.

### Summary: Multiple imputation

* Procedure:
    - Apply probabilistic imputation multiple times, resulting in $m$ imputed datasets.
    - Compute statistics separately on the $m$ imputed datasets (e.g. compute the mean or correlation coefficient).
    - Plot the distribution of these statistics and create confidence intervals.
* If a column is missing conditional on multiple columns, your "multiple imputations" should include probabilistic imputations for each!